In [1]:
import numpy as np 
import tensorflow as tf

#load data set
x_real = np.loadtxt("train_x.csv", delimiter=",") # load from text 
y_real = np.loadtxt("train_y.csv", delimiter=",") 
#x = x.reshape(-1, 64, 64) # reshape 
#y = y.reshape(-1, 1) 

In [21]:
x_real = np.where(x_real > 252, 1, 0)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 54, 56, 63, 64, 72, 81]
# lb = preprocessing.LabelBinarizer()
# lb.fit(classes)


x_train, x_test, y_train, y_test = train_test_split(x_real[:10000], y_real[:10000], test_size=0.25, random_state=42)

#normalize value in x_train


In [24]:
#Convolutional Neural Network using Tensorflow
#labels is [0,1,2,....81]

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

def cnn_model_fn(features, labels, mode):
    
    #Model function for CNN.
    # Input Layer
    # [batch_size, image_width, image_height, channels]
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])

    # Convolutional Layer #1
    # Input Tensor Shape: [batch_size, 64, 64, 1]
    # Output Tensor Shape: [batch_size, 64, 64, 32]
    conv1 = tf.layers.conv2d(inputs=input_layer,
                             filters=32,
                             kernel_size=[5, 5],
                             padding="same",
                             activation=tf.nn.relu)

    # Pooling Layer #1
    # Input Tensor Shape: [batch_size, 64, 64, 32]
    # Output Tensor Shape: [batch_size, 32, 32, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                    pool_size=[2, 2],
                                    strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    # Input Tensor Shape: [batch_size, 32, 32, 32]
    # Output Tensor Shape: [batch_size, 32, 32, 64]
    conv2 = tf.layers.conv2d(inputs=pool1,
                             filters=64,
                             kernel_size=[5, 5],
                             padding="same",
                             activation=tf.nn.relu)
    # Input Tensor Shape: [batch_size, 32, 32, 64]
    # Output Tensor Shape: [batch_size, 16, 16, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2,
                                    pool_size=[2, 2],
                                    strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    # Dropout to prevent overfitting
    # Output Tensor Shape: [batch_size, 1024]
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=40)

    # Generate predictions (for PREDICT and EVAL mode)
    # Add `softmax_tensor` to the graph. It is used for PREDICT and by the `logging_hook`.
      
    predictions = {"classes": tf.argmax(input=logits, axis=1),
                   "probabilities": tf.nn.softmax(logits, name="softmax_tensor")}

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=40)
    
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [25]:
#Create estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,model_dir="/tmp/comp551_cnn_model3")

#Set up Logging Hook
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

#Start training model
x_train = np.float32(x_train)
x_test = np.float32(x_test)
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":x_train},y=y_train,batch_size=100,num_epochs=None,shuffle=True)
mnist_classifier.train(input_fn=train_input_fn,steps=20000,hooks=[logging_hook])

#Start evaluating model 
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":x_test},y=y_test,num_epochs=1,shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)
                                                   
                                                   
                                                   

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_master': '', '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_is_chief': True, '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000011F122126D8>, '_model_dir': '/tmp/comp551_cnn_model3', '_task_type': 'worker', '_save_checkpoints_secs': 600}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/comp551_cnn_model3\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.025  0.025  0.025 ...,  0.025  0.025  0.025]
 [ 0.025  0.025  0.025 ...,  0.025  0.025  0.025]
 [ 0.025  0.025  0.025 ...,  0.025  0.025  0.025]
 ..., 
 [ 0.025  0.025  0.025 ...,  0.025  0.025  0.025]
 [ 0.025  0.025  0.025 ...,  0.025  0.025  0.

INFO:tensorflow:step = 1201, loss = 2.89012 (3.663 sec)
INFO:tensorflow:global_step/sec: 27.4581
INFO:tensorflow:probabilities = [[ 0.12012526  0.01589688  0.02761847 ...,  0.00597324  0.00597324
   0.00597324]
 [ 0.12012526  0.01589688  0.02761847 ...,  0.00597324  0.00597324
   0.00597324]
 [ 0.12012526  0.01589688  0.02761847 ...,  0.00597324  0.00597324
   0.00597324]
 ..., 
 [ 0.12012526  0.01589688  0.02761847 ...,  0.00597324  0.00597324
   0.00597324]
 [ 0.12012526  0.01589688  0.02761847 ...,  0.00597324  0.00597324
   0.00597324]
 [ 0.12012526  0.01589688  0.02761847 ...,  0.00597324  0.00597324
   0.00597324]] (3.642 sec)
INFO:tensorflow:step = 1301, loss = 2.83531 (3.643 sec)
INFO:tensorflow:global_step/sec: 27.1342
INFO:tensorflow:probabilities = [[ 0.1200129   0.01595618  0.02778297 ...,  0.00572437  0.00572437
   0.00572437]
 [ 0.1200129   0.01595618  0.02778297 ...,  0.00572437  0.00572437
   0.00572437]
 [ 0.1200129   0.01595618  0.02778297 ...,  0.00572437  0.00572437

INFO:tensorflow:step = 2501, loss = 2.69567 (3.645 sec)
INFO:tensorflow:global_step/sec: 27.1888
INFO:tensorflow:probabilities = [[ 0.12233321  0.01650121  0.02912908 ...,  0.00407247  0.00407247
   0.00407247]
 [ 0.12233321  0.01650121  0.02912908 ...,  0.00407247  0.00407247
   0.00407247]
 [ 0.12233321  0.01650121  0.02912908 ...,  0.00407247  0.00407247
   0.00407247]
 ..., 
 [ 0.12233321  0.01650121  0.02912908 ...,  0.00407247  0.00407247
   0.00407247]
 [ 0.12233321  0.01650121  0.02912908 ...,  0.00407247  0.00407247
   0.00407247]
 [ 0.12233321  0.01650121  0.02912908 ...,  0.00407247  0.00407247
   0.00407247]] (3.670 sec)
INFO:tensorflow:step = 2601, loss = 2.75889 (3.674 sec)
INFO:tensorflow:global_step/sec: 27.4587
INFO:tensorflow:probabilities = [[ 0.1193504   0.01649005  0.02919841 ...,  0.00400344  0.00400344
   0.00400344]
 [ 0.1193504   0.01649005  0.02919841 ...,  0.00400344  0.00400344
   0.00400344]
 [ 0.1193504   0.01649005  0.02919841 ...,  0.00400344  0.00400344

INFO:tensorflow:step = 3801, loss = 2.82758 (3.674 sec)
INFO:tensorflow:global_step/sec: 27.0744
INFO:tensorflow:probabilities = [[ 0.12182868  0.0167335   0.02938347 ...,  0.00339372  0.00339372
   0.00339372]
 [ 0.12182868  0.0167335   0.02938347 ...,  0.00339372  0.00339372
   0.00339372]
 [ 0.12182868  0.0167335   0.02938347 ...,  0.00339372  0.00339372
   0.00339372]
 ..., 
 [ 0.12182868  0.0167335   0.02938347 ...,  0.00339372  0.00339372
   0.00339372]
 [ 0.12182868  0.0167335   0.02938347 ...,  0.00339372  0.00339372
   0.00339372]
 [ 0.12182868  0.0167335   0.02938347 ...,  0.00339372  0.00339372
   0.00339372]] (3.694 sec)
INFO:tensorflow:step = 3901, loss = 2.74145 (3.692 sec)
INFO:tensorflow:global_step/sec: 27.1894
INFO:tensorflow:probabilities = [[ 0.11954696  0.01687197  0.02950683 ...,  0.00336537  0.00336537
   0.00336537]
 [ 0.11954696  0.01687197  0.02950683 ...,  0.00336537  0.00336537
   0.00336537]
 [ 0.11954696  0.01687197  0.02950683 ...,  0.00336537  0.00336537

INFO:tensorflow:step = 5101, loss = 2.71229 (3.658 sec)
INFO:tensorflow:global_step/sec: 27.0602
INFO:tensorflow:probabilities = [[ 0.12133256  0.01693243  0.02952465 ...,  0.00306576  0.00306576
   0.00306576]
 [ 0.12133256  0.01693243  0.02952465 ...,  0.00306576  0.00306576
   0.00306576]
 [ 0.12133256  0.01693243  0.02952465 ...,  0.00306576  0.00306576
   0.00306576]
 ..., 
 [ 0.12133256  0.01693243  0.02952465 ...,  0.00306576  0.00306576
   0.00306576]
 [ 0.12133256  0.01693243  0.02952465 ...,  0.00306576  0.00306576
   0.00306576]
 [ 0.12133256  0.01693243  0.02952465 ...,  0.00306576  0.00306576
   0.00306576]] (3.693 sec)
INFO:tensorflow:step = 5201, loss = 3.07425 (3.694 sec)
INFO:tensorflow:global_step/sec: 27.2738
INFO:tensorflow:probabilities = [[ 0.12055304  0.01707543  0.02935481 ...,  0.00305233  0.00305233
   0.00305233]
 [ 0.12055304  0.01707543  0.02935481 ...,  0.00305233  0.00305233
   0.00305233]
 [ 0.12055304  0.01707543  0.02935481 ...,  0.00305233  0.00305233

...

INFO:tensorflow:step = 14201, loss = 2.72396 (3.677 sec)
INFO:tensorflow:global_step/sec: 27.1927
INFO:tensorflow:probabilities = [[ 0.11971372  0.01722907  0.02977436 ...,  0.00263332  0.00263332
   0.00263332]
 [ 0.11971372  0.01722907  0.02977436 ...,  0.00263332  0.00263332
   0.00263332]
 [ 0.11971372  0.01722907  0.02977436 ...,  0.00263332  0.00263332
   0.00263332]
 ..., 
 [ 0.11971372  0.01722907  0.02977436 ...,  0.00263332  0.00263332
   0.00263332]
 [ 0.11971372  0.01722907  0.02977436 ...,  0.00263332  0.00263332
   0.00263332]
 [ 0.11971372  0.01722907  0.02977436 ...,  0.00263332  0.00263332
   0.00263332]] (3.669 sec)
INFO:tensorflow:step = 14301, loss = 2.7945 (3.673 sec)
INFO:tensorflow:global_step/sec: 27.1552
INFO:tensorflow:probabilities = [[ 0.12152495  0.01715674  0.02960045 ...,  0.00262737  0.00262737
   0.00262737]
 [ 0.12152495  0.01715674  0.02960045 ...,  0.00262737  0.00262737
   0.00262737]
 [ 0.12152495  0.01715674  0.02960045 ...,  0.00262737  0.0026273

INFO:tensorflow:step = 15501, loss = 2.79554 (3.696 sec)
INFO:tensorflow:global_step/sec: 27.0166
INFO:tensorflow:probabilities = [[ 0.1219604   0.01711518  0.02974426 ...,  0.00262208  0.00262208
   0.00262208]
 [ 0.1219604   0.01711518  0.02974426 ...,  0.00262208  0.00262208
   0.00262208]
 [ 0.1219604   0.01711518  0.02974426 ...,  0.00262208  0.00262208
   0.00262208]
 ..., 
 [ 0.1219604   0.01711518  0.02974426 ...,  0.00262208  0.00262208
   0.00262208]
 [ 0.1219604   0.01711518  0.02974426 ...,  0.00262208  0.00262208
   0.00262208]
 [ 0.1219604   0.01711518  0.02974426 ...,  0.00262208  0.00262208
   0.00262208]] (3.701 sec)
INFO:tensorflow:step = 15601, loss = 2.74434 (3.696 sec)
INFO:tensorflow:global_step/sec: 27.1916
INFO:tensorflow:probabilities = [[ 0.11955059  0.01711169  0.02966302 ...,  0.0026222   0.0026222
   0.0026222 ]
 [ 0.11955059  0.01711169  0.02966302 ...,  0.0026222   0.0026222
   0.0026222 ]
 [ 0.11955059  0.01711169  0.02966302 ...,  0.0026222   0.0026222


INFO:tensorflow:step = 16801, loss = 2.80254 (3.688 sec)
INFO:tensorflow:global_step/sec: 27.1381
INFO:tensorflow:probabilities = [[ 0.11961356  0.01720486  0.02994061 ...,  0.00262233  0.00262233
   0.00262233]
 [ 0.11961356  0.01720486  0.02994061 ...,  0.00262233  0.00262233
   0.00262233]
 [ 0.11961356  0.01720486  0.02994061 ...,  0.00262233  0.00262233
   0.00262233]
 ..., 
 [ 0.11961356  0.01720486  0.02994061 ...,  0.00262233  0.00262233
   0.00262233]
 [ 0.11961356  0.01720486  0.02994061 ...,  0.00262233  0.00262233
   0.00262233]
 [ 0.11961356  0.01720486  0.02994061 ...,  0.00262233  0.00262233
   0.00262233]] (3.677 sec)
INFO:tensorflow:step = 16901, loss = 2.94619 (3.678 sec)
INFO:tensorflow:global_step/sec: 27.135
INFO:tensorflow:probabilities = [[ 0.12176415  0.0171392   0.02982342 ...,  0.00261901  0.00261901
   0.00261901]
 [ 0.12176415  0.0171392   0.02982342 ...,  0.00261901  0.00261901
   0.00261901]
 [ 0.12176415  0.0171392   0.02982342 ...,  0.00261901  0.0026190

INFO:tensorflow:step = 18101, loss = 2.80061 (3.706 sec)
INFO:tensorflow:global_step/sec: 27.0771
INFO:tensorflow:probabilities = [[ 0.12102901  0.01713465  0.02971557 ...,  0.00261671  0.00261671
   0.00261671]
 [ 0.12102901  0.01713465  0.02971557 ...,  0.00261671  0.00261671
   0.00261671]
 [ 0.12102901  0.01713465  0.02971557 ...,  0.00261671  0.00261671
   0.00261671]
 ..., 
 [ 0.12102901  0.01713465  0.02971557 ...,  0.00261671  0.00261671
   0.00261671]
 [ 0.12102901  0.01713465  0.02971557 ...,  0.00261671  0.00261671
   0.00261671]
 [ 0.12102901  0.01713465  0.02971557 ...,  0.00261671  0.00261671
   0.00261671]] (3.685 sec)
INFO:tensorflow:step = 18201, loss = 2.73432 (3.687 sec)
INFO:tensorflow:global_step/sec: 27.1323
INFO:tensorflow:probabilities = [[ 0.12080291  0.01708923  0.02966896 ...,  0.00261541  0.00261541
   0.00261541]
 [ 0.12080291  0.01708923  0.02966896 ...,  0.00261541  0.00261541
   0.00261541]
 [ 0.12080291  0.01708923  0.02966896 ...,  0.00261541  0.002615

INFO:tensorflow:step = 19401, loss = 2.88288 (3.674 sec)
INFO:tensorflow:global_step/sec: 26.7474
INFO:tensorflow:probabilities = [[ 0.12155198  0.01712776  0.029642   ...,  0.00261456  0.00261456
   0.00261456]
 [ 0.12155198  0.01712776  0.029642   ...,  0.00261456  0.00261456
   0.00261456]
 [ 0.12155198  0.01712776  0.029642   ...,  0.00261456  0.00261456
   0.00261456]
 ..., 
 [ 0.12155198  0.01712776  0.029642   ...,  0.00261456  0.00261456
   0.00261456]
 [ 0.12155198  0.01712776  0.029642   ...,  0.00261456  0.00261456
   0.00261456]
 [ 0.12155198  0.01712776  0.029642   ...,  0.00261456  0.00261456
   0.00261456]] (3.731 sec)
INFO:tensorflow:step = 19501, loss = 2.69501 (3.733 sec)
INFO:tensorflow:global_step/sec: 27.0963
INFO:tensorflow:probabilities = [[ 0.11977556  0.0171747   0.0300033  ...,  0.00261972  0.00261972
   0.00261972]
 [ 0.11977556  0.0171747   0.0300033  ...,  0.00261972  0.00261972
   0.00261972]
 [ 0.11977556  0.0171747   0.0300033  ...,  0.00261972  0.002619